In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import pycountry

In [2]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

In [3]:
def mkdf(filename):
    df = pd.read_csv(filename)
    cols = df.columns
    data = df.loc[:, cols[1:]].sum(axis=1).tolist()
    total = data.pop(-1)
    locations = df["ctry"].tolist()
    locations.pop(-1)
    for i,location in enumerate(locations):
        try:
            if location == pycountry.countries.get(alpha_3=location).alpha_3:
                # print(f"{location} is alpha3, skipping.")
                continue
        except AttributeError:
            # print(f"{i}, {location}")
            pass
        try:
            locations[i] = pycountry.countries.get(name=location).alpha_3
        except AttributeError:
            print(f"No name for {location}.")
            try:
                locations[i] = pycountry.countries.get(official_name=location).alpha_3
                pass
            except AttributeError:
                print(f"No official name for {location} either, Duh...")

    dict = {"Country": locations, "Frequency": data}
    out = pd.DataFrame(dict).groupby(['Country'],as_index=False).agg({'Frequency': 'sum'})
    # data.groupby(['Country'],as_index=False).agg({'Frequency': 'sum'})
    return out

In [4]:
africa = mkdf("africa_dataset.csv")
asia = mkdf("asia_dataset.csv")
europe = mkdf("europe_dataset.csv")
latin_america = mkdf("latin_america_dataset.csv")
middle_east = mkdf("middle_east_dataset.csv")
north_america = mkdf("north_america_dataset.csv")
oceania = mkdf("oceania_dataset.csv")

In [5]:
data = pd.concat([africa, asia, europe, latin_america, middle_east, north_america, oceania])

In [32]:
mycolor = [
    "cornsilk",
    "bisque",
    "goldenrod",
    "peru",
    "chocolate",
    "saddlebrown",
    "maroon"
]
# projections: https://etpinard.github.io/plotly-dashboards/map-projections/
fig = px.choropleth(data, locations='Country', locationmode="ISO-3", color="Frequency", hover_name='Country', projection='natural earth', title=None, color_continuous_scale=mycolor)
# fig.show()
fig.write_image("images/fig1.svg")
# fig.write_image("images/fig1.jpg")